In [98]:
import numpy as np
from typing import Dict, Tuple, Sequence, List, Any
import matplotlib.pyplot as plt
import folium
import pandas as pd 
import datetime


In [99]:
df = pd.read_csv("dhl-data.csv")

df.describe()
# df = df.drop(['ShpCnt', 'Customer Name', 'City', 'Street', 'zip', 'Pallets Pcs', 'Total Pcs' , 'Parcel Pcs', 'Weight' , 'PUD Svc Area', 'PUD Fac', 'Courier Type','awb_booking', 'Stop Code', 'MarkerColor'], 1)

# hardly useful for our scenario
df = df.drop(['ShpCnt', 'Pallets Pcs', 'Total Pcs' , 'Parcel Pcs', 'Weight', 'Courier Type', 'Stop Code', 'MarkerColor'], 1)

df = df.drop("awb_booking",1)

df = df.drop(['Customer Name', 'City', 'Street'], 1)

# sure 20200116, no meaning
df = df.drop(['Act Dt'], 1)

# sure EPJ, no meaning
df = df.drop(['PUD Svc Area'], 1)

# sure KUL, no meaning
df = df.drop(['PUD Fac'], 1)

# sure KUL, no meaning
df = df.drop(['Delivery Type', 'Pickup Type'], 1)

#remove no lat lon, cause no planning create geo code
df = df.dropna(subset = ['lat', 'lgtd'])

# remove column cant know pick or delivery
df = df.dropna(subset = ['Act Base'])

# df = df[df["Delivery Type"] != 'NaN']
# df = df.isnull(df["Delivery Type"])

# df["Pickup Type"].dropna() == df["AR dtm"].dropna()


df
# df.describe()

,PUD Rte,PUD Cycle,Courier id,zip,Act Base,Prod Grp,Prod Code,AR dtm,Act Tm,Open,Closed,lat,lgtd,Act Ckpt Code,PuD Type
1,PJA1,B,goramali,40470.0,D,DOX,D,1/16/2020 9:03,10:44,NaN,23:59,3.017103,101.523414,OK,DL
2,PJA1,B,goramali,40400.0,D,DOX,D,1/16/2020 9:04,10:50,NaN,23:59,3.012378,101.521595,OK,DL
3,PJA1,B,goramali,40460.0,D,DOM,N,1/15/2020 21:46,10:58,NaN,23:59,3.005459,101.516529,OK,DL
4,PJA1,B,goramali,40460.0,D,WPX,P,1/16/2020 1:29,11:00,NaN,23:59,3.005232,101.516094,OK,DL
5,PJA1,B,goramali,40460.0,D,DOM,N,1/15/2020 22:35,11:04,NaN,23:59,3.007725,101.514426,OK,DL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,PJR8,B,tverasam,46050.0,D,DOM,N,1/15/2020 21:51,15:31,NaN,23:59,3.088077,101.636774,OK,DL
4202,PJR8,B,tverasam,47301.0,P,WPX,P,NaN,16:21,9:00,16:30,3.098278,101.590949,PU,PU
4203,PJR8,B,tverasam,68100.0,D,WPX,P,1/16/2020 1:24,17:30,NaN,23:59,3.233742,101.680733,OK,DL
4204,PJR8,B,tverasam,47810.0,P,WPX,P,NaN,18:04,11:15,16:30,3.156957,101.565938,PU,PU


In [100]:
# print(f"Missing values in each variable: \n{df.isnull().sum()}")
# print(f"Variable:                  Type: \n{df.dtypes}") 
# print(f"Duplicated rows: {df.duplicated().sum()}")

In [101]:
# df.duplicated()

In [102]:
df

,PUD Rte,PUD Cycle,Courier id,zip,Act Base,Prod Grp,Prod Code,AR dtm,Act Tm,Open,Closed,lat,lgtd,Act Ckpt Code,PuD Type
1,PJA1,B,goramali,40470.0,D,DOX,D,1/16/2020 9:03,10:44,NaN,23:59,3.017103,101.523414,OK,DL
2,PJA1,B,goramali,40400.0,D,DOX,D,1/16/2020 9:04,10:50,NaN,23:59,3.012378,101.521595,OK,DL
3,PJA1,B,goramali,40460.0,D,DOM,N,1/15/2020 21:46,10:58,NaN,23:59,3.005459,101.516529,OK,DL
4,PJA1,B,goramali,40460.0,D,WPX,P,1/16/2020 1:29,11:00,NaN,23:59,3.005232,101.516094,OK,DL
5,PJA1,B,goramali,40460.0,D,DOM,N,1/15/2020 22:35,11:04,NaN,23:59,3.007725,101.514426,OK,DL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,PJR8,B,tverasam,46050.0,D,DOM,N,1/15/2020 21:51,15:31,NaN,23:59,3.088077,101.636774,OK,DL
4202,PJR8,B,tverasam,47301.0,P,WPX,P,NaN,16:21,9:00,16:30,3.098278,101.590949,PU,PU
4203,PJR8,B,tverasam,68100.0,D,WPX,P,1/16/2020 1:24,17:30,NaN,23:59,3.233742,101.680733,OK,DL
4204,PJR8,B,tverasam,47810.0,P,WPX,P,NaN,18:04,11:15,16:30,3.156957,101.565938,PU,PU


In [103]:
df.to_csv("processed_dhl_data.csv")

In [116]:
from datetime import datetime

# morning_parcel = "01/15/2020 00:00"
# until_morning_parcel = "01/16/2020 09:15"

# afternoon_parcel = "01/16/2020 09:16"
# until_afternoon_parcel = "01/16/2020 1:15"

morning_parcel = "2020/1/15 00:00"
until_morning_parcel = "2020/1/16 09:15"

afternoon_parcel = "2020/1/15 09:16"
until_afternoon_parcel = "2020/1/16 1:15"

# morning_parcel_dt = datetime.strptime(morning_parcel, "%m/%d/%Y %H:%M")
# until_morning_parcel_dt = datetime.strptime(until_morning_parcel, "%m/%d/%Y %H:%M")
# afternoon_parcel_dt = datetime.strptime(afternoon_parcel, "%m/%d/%Y %H:%M")
# until_afternoon_parcel_dt = datetime.strptime(until_afternoon_parcel, "%m/%d/%Y %H:%M")

#df = df.loc[datetime.strptime(df["AR dtm"], "%m/%d/%Y %H:%M")  ]

delivery = df.loc[df['Act Base'] == "D"]
pickup = df.loc[df['Act Base'] == "P"]

#no arrival time and delivery task can not be use
delivery = df.dropna(subset = ['AR dtm'])

bigdata = delivery.append(pickup, ignore_index=True)

delivery['AR dtm'] =  pd.to_datetime(delivery['AR dtm'], format='%m/%d/%Y %H:%M')

mask = (delivery['AR dtm'] >= morning_parcel) & (delivery['AR dtm'] <= until_morning_parcel)
morning = delivery.loc[mask]
afternoon = delivery.loc[~mask]


morning
#afternoon

D:\Users\MX-15\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,PUD Rte,PUD Cycle,Courier id,zip,Act Base,Prod Grp,Prod Code,AR dtm,Act Tm,Open,Closed,lat,lgtd,Act Ckpt Code,PuD Type
1,PJA1,B,goramali,40470.0,D,DOX,D,2020-01-16 09:03:00,10:44,NaN,23:59,3.017103,101.523414,OK,DL
2,PJA1,B,goramali,40400.0,D,DOX,D,2020-01-16 09:04:00,10:50,NaN,23:59,3.012378,101.521595,OK,DL
3,PJA1,B,goramali,40460.0,D,DOM,N,2020-01-15 21:46:00,10:58,NaN,23:59,3.005459,101.516529,OK,DL
4,PJA1,B,goramali,40460.0,D,WPX,P,2020-01-16 01:29:00,11:00,NaN,23:59,3.005232,101.516094,OK,DL
5,PJA1,B,goramali,40460.0,D,DOM,N,2020-01-15 22:35:00,11:04,NaN,23:59,3.007725,101.514426,OK,DL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,PJR8,B,tverasam,46100.0,D,WPX,P,2020-01-16 01:27:00,12:21,NaN,23:59,3.102691,101.633487,OK,DL
4198,PJR8,B,tverasam,46050.0,D,WPX,P,2020-01-16 01:29:00,12:59,NaN,23:59,3.101048,101.640262,OK,DL
4199,PJR8,B,tverasam,46050.0,D,WPX,P,2020-01-16 01:31:00,15:29,NaN,23:59,3.088077,101.636774,OK,DL
4200,PJR8,B,tverasam,46050.0,D,DOM,N,2020-01-15 21:51:00,15:31,NaN,23:59,3.088077,101.636774,OK,DL


In [ ]:
d